In [ ]:
# Import libraries
from pybis import Openbis
from IPython.display import display, HTML, Javascript
import ipywidgets as widgets
import json
import os

In [ ]:
# Global variables
OPENBIS_SESSION = None

In [ ]:
# Widgets
openbis_connection_status_htmlbox = widgets.HTML(value = '')
openbis_url_textbox = widgets.Text(value = 'openbis', placeholder = 'Enter openBIS URL', description = 'openBIS URL:', disabled = False)
openbis_url_textbox.style = {'description_width': 'initial'}
username_textbox = widgets.Text(value = 'admin', placeholder = 'Enter username', description = 'Username:', disabled = False)
password_textbox = widgets.Password(value = '123456789', placeholder = 'Enter password', description = 'Password:', disabled = False)

connect_button = widgets.Button(description = 'Connect', disabled = False, button_style = 'success', tooltip = 'Connect with openBIS', layout = widgets.Layout(width='200px'))
connect_button.style = {'description_width': 'initial'}

disconnect_button = widgets.Button(description = 'Disconnect', disabled = True, button_style = 'danger', tooltip = 'Disconnect openBIS', layout = widgets.Layout(width='200px'))
disconnect_button.style = {'description_width': 'initial'}
disconnect_button.add_class = disconnect_button

connection_buttons = widgets.HBox([connect_button, disconnect_button])

open_notebooks_html_disable_code = '''
<div id="box" style="display: flex; pointer-events: none;">
    <div style="flex: 1; padding: 10px;">
        <ul>
            <li><a href="create_sample.ipynb" style="color: gray;" target="_blank">Create a new sample</a></li>
            <li><a href="sputtering.ipynb" style="color: gray;" target="_blank">Sputter sample</a></li>
            <li><a href="annealing.ipynb" style="color: gray;" target="_blank">Anneal sample</a></li>
        </ul>
    </div>
    <div style="flex: 1; padding: 10px;">
        <ul>
            <li><a href="deposition.ipynb" style="color: gray;" target="_blank">Deposit a molecule on a sample</a></li>
            <li><a href="measurement.ipynb" style="color: gray;" target="_blank">Measure a sample</a></li>
        </ul>
    </div>
</div>
'''

open_notebooks_html_enable_code = '''
    <div style="display: flex;">
        <div style="flex: 1; padding: 10px;">
            <ul>
                <li><a href="create_sample.ipynb" target="_blank">Create a new sample</a></li>
                <li><a href="sputtering.ipynb" target="_blank">Sputter sample</a></li>
                <li><a href="annealing.ipynb" target="_blank">Anneal sample</a></li>
            </ul>
        </div>
        <div style="flex: 1; padding: 10px;">
            <ul>
                <li><a href="deposition.ipynb" target="_blank">Deposit a molecule on a sample</a></li>
                <li><a href="measurement.ipynb" target="_blank">Measure a sample</a></li>
            </ul>
        </div>
    </div>
    '''
open_notebooks_htmlbox = widgets.HTML(open_notebooks_html_disable_code)

In [ ]:
# Functions
def log_in(bisurl='openbis', bisuser='admin', bispasswd='changeit'):
    """Function to login to openBIS."""
    if Openbis(bisurl, verify_certificates=False).is_token_valid():
        session = Openbis(bisurl, verify_certificates=False)
    else:
        Openbis(bisurl, verify_certificates=False).login(bisuser, bispasswd, save_token=True)
        session = Openbis(bisurl, verify_certificates=False)
    return session

def create_json(json_content, filename):
    with open(filename, 'w') as file:
        json.dump(json_content, file, indent=4)

def read_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def connect_to_openbis(b):
    global OPENBIS_SESSION
    
    if OPENBIS_SESSION is None or OPENBIS_SESSION.is_session_active() == False:
        OPENBIS_SESSION = log_in(openbis_url_textbox.value, username_textbox.value, password_textbox.value)
        json_content = {"url": OPENBIS_SESSION.url, 
                        "token": OPENBIS_SESSION.token}
        create_json(json_content, "token.json")
        openbis_connection_status_htmlbox.value = "<span style='color: green; font-weight: bold;'>Connected to openBIS!</span>"
        open_notebooks_htmlbox.value = open_notebooks_html_enable_code
        connect_button.disabled = True
        disconnect_button.disabled = False

def disconnect_openbis(b):
    global OPENBIS_SESSION
    
    if OPENBIS_SESSION.is_session_active():
        OPENBIS_SESSION.logout()
        openbis_connection_status_htmlbox.value = "<span style='color: red; font-weight: bold;'>Please connect to openBIS!</span>"
        os.remove("token.json")
        open_notebooks_htmlbox.value = open_notebooks_html_disable_code
        connect_button.disabled = False
        disconnect_button.disabled = True

# Jupyter notebook app to save experiment information in openBIS

## Create connection with openBIS

In [ ]:
if os.path.exists("token.json"):
    openbis_token = read_json("token.json")
    # When session is no longer valid.
    try:
        OPENBIS_SESSION = Openbis(openbis_token["url"], verify_certificates = False)
        OPENBIS_SESSION.set_token(openbis_token["token"])
    except ValueError:
        OPENBIS_SESSION = None
        os.remove("token.json")

if OPENBIS_SESSION is None:
    openbis_connection_status_htmlbox.value = "<span style='color: red; font-weight: bold;'>Please connect to openBIS!</span>"
else:
    openbis_connection_status_htmlbox.value = "<span style='color: green; font-weight: bold;'>Connected to openBIS!</span>"
    open_notebooks_htmlbox.value = open_notebooks_html_enable_code
    connect_button.disabled = True
    disconnect_button.disabled = False
    
display(widgets.VBox([openbis_connection_status_htmlbox, 
                      widgets.HBox([openbis_url_textbox, 
                                    username_textbox, 
                                    password_textbox]), 
                      connection_buttons],
                    layout = widgets.Layout(width='100%', border='solid 1px black')
))
connect_button.on_click(connect_to_openbis)
disconnect_button.on_click(disconnect_openbis)

## Work on a sample object

In [ ]:
display(widgets.VBox([open_notebooks_htmlbox], 
                     layout = widgets.Layout(width='100%', border='solid 1px black')
))
